In [1]:
import cvxpy as cvx
import numpy as np

### Setup Constants

In [2]:
l = 0.98
g = 9.8
m = 0.825
M = 8.085

tol = 1e-8
i = 0
MAX_ITER = 25

$\alpha$ for bisection

In [3]:
alpha_up = 800
alpha_lo = 0

### Dynamics

In [5]:
A = np.array([[0, 1, 0, 0], [0, 0, -(m*g/M), 0], [0, 0, 0, 1], [0, 0, g/l, 0]])
B = np.array([[0], 
              [1/M],
              [0],
              [-1/(M*l)]])
I = np.eye(4)

In [28]:
Q = cvx.Semidef(4)
Z = cvx.Variable(1,4)
obj = cvx.Minimize(0)

### Change of Variables for the Lyapunov Function
A change of variables is required to allow for a $K$ to be found.
$$ A_{cl}^TP + PA_{cl} + 2\alpha P < 0 $$
$$ (A + BK)^TP + P(A + BK) + 2\alpha P < 0 $$
$$ A^TP + K^TB^TP + PA + PBK + 2\alpha P < 0 $$

However, it is impossible to find a K from the above equation and so pre- and post-multiply by $P^{-1}$. 
$$ P^{-1}A^T + P^{-1}K^TB^T + AP^{-1} + BKP^{-1} + 2\alpha P^{-1} < 0 $$

And set $Q = P^{-1}$ and $Z = KP^{-1}$. Which becomes:
$$ QA^T + Z^TB^T + AQ + BZ + 2\alpha Q < 0 $$

In [11]:
while(abs(alpha_up - alpha_lo) > tol and i < MAX_ITER):
    i += 1
    alpha = 0.5*(alpha_up + alpha_lo)
    const = [Q*A.T + (B*Z).T + A*Q + B*Z + 2*alpha*Q < -I, Q > I]
    
    prob = cvx.Problem(obj, const)
    prob.solve()
    
    if prob.status == "infeasible":
        alpha_up = alpha
    else:
        alpha_lo = alpha

In [12]:
alpha

9.5367431640625e-05

In [17]:
print(Z.value)

None


In [18]:
Z

Variable(1, 4)

In [26]:
alpha = 0.
const = [Q*A.T + (B*Z).T + A*Q + B*Z + 2*alpha*Q < -I, Q > I]

In [27]:
prob = cvx.Problem(obj, const)
prob.solve(verbose=True)

DCPError: Problem does not follow DCP rules.